In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data=pd.read_csv('tmdb_5000_movies.csv')
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [3]:
data.shape

(4803, 20)

In [4]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [5]:
data = data[['id','genres', 'vote_average', 'vote_count','popularity','title',  'keywords', 'overview']]

In [6]:
tmp_m=data['vote_count'].quantile(0.89)
tmp_m

1683.8999999999987

In [7]:
tmp_data = data.copy().loc[data['vote_count'] >= tmp_m]
tmp_data.shape

(529, 8)

In [8]:
del tmp_data

m=data['vote_count'].quantile(0.9)
data=data.loc[data['vote_count']>=m]

In [9]:
C=data['vote_average'].mean()

In [10]:
print(C)
print(m)

6.9629937629937615
1838.4000000000015


In [11]:
def weighted_rating(x,m=m,C=C):
    v=x['vote_count']
    R=x['vote_average']
    
    return (v/(v+m)*R) + (m/(m+v)*C)

In [12]:
data['score']=data.apply(weighted_rating,axis=1)

In [13]:
data.head()

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",6.918271
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,6.493333
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,7.492998
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",6.500396


In [14]:
data.shape

(481, 9)

In [15]:
data[['genres','keywords']].head(2)

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."


In [16]:
data['genres']=data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

In [17]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."


In [18]:
data['genres']=data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join)
data['keywords'] = data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [19]:
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,<built-in method join of str object at 0x00000...,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,<built-in method join of str object at 0x00000...,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.918271


In [21]:
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,<built-in method join of str object at 0x00000...,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,<built-in method join of str object at 0x00000...,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.918271


In [20]:
data.to_csv('pre_tmdb_5000_movies.csv', index = False)

In [22]:
movie_data=pd.read_csv('movies_metadata.csv')
movie_data =  movie_data.loc[movie_data['original_language'] == 'en', :]
movie_data = movie_data[['id', 'title', 'original_language', 'genres']]

print(movie_data.shape)
movie_data.head()

(32269, 4)


C:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,title,original_language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"


In [23]:
movie_keyword = pd.read_csv('keywords.csv')
print(movie_keyword.shape)
movie_keyword.head()

(46419, 2)


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [24]:
movie_data.id = movie_data.id.astype(int)
movie_keyword.id = movie_keyword.id.astype(int)
movie_data = pd.merge(movie_data, movie_keyword, on='id')
print(movie_data.shape)
movie_data.head()

(32852, 5)


,id,title,original_language,genres,keywords
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [25]:
movie_data['genres'] = movie_data['genres'].apply(literal_eval)
movie_data['genres'] = movie_data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [26]:

movie_data['keywords'] = movie_data['keywords'].apply(literal_eval)
movie_data['keywords'] = movie_data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [27]:

movie_data.head()

,id,title,original_language,genres,keywords
0,862,Toy Story,en,Animation Comedy Family,jealousy toy boy friendship friends rivalry bo...
1,8844,Jumanji,en,Adventure Fantasy Family,board game disappearance based on children's b...
2,15602,Grumpier Old Men,en,Romance Comedy,fishing best friend duringcreditsstinger old men
3,31357,Waiting to Exhale,en,Comedy Drama Romance,based on novel interracial relationship single...
4,11862,Father of the Bride Part II,en,Comedy,baby midlife crisis confidence aging daughter ...


In [28]:
tfidf_vector = TfidfVectorizer()
#tfidf_vector = TfidfVectorizer(ngram_range=(1,2))
tfidf_matrix = tfidf_vector.fit_transform(movie_data['genres'] + " " + movie_data['keywords']).toarray()
#tfidf_matrix = tfidf_vector.fit_transform(movie_data['genres']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names()

In [29]:
tfidf_matrix.shape

(32852, 11437)

In [30]:

tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = movie_data.title)
print(tfidf_matrix.shape)
tfidf_matrix.head()

(32852, 11437)


,077,10,11,13,1500s,15th,16th,17th,1812,18th,...,βάφτηκε,γη,κόκκινο,το,χώμα,миньоны,卧底肥妈,绝地奶霸,自然界大事件,超级妈妈
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jumanji,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grumpier Old Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Waiting to Exhale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Father of the Bride Part II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
%%time
cosine_sim = cosine_similarity(tfidf_matrix)

Wall time: 2min 4s


In [32]:
cosine_sim.shape

(32852, 32852)

In [33]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = movie_data.title, columns = movie_data.title)
print(cosine_sim_df.shape)
cosine_sim_df.head()

(32852, 32852)


title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Deep Hearts,The Morning After,House of Horrors,Shadow of the Blair Witch,The Burkittsville 7,Caged Heat 3000,Robin Hood,Betrayal,Satan Triumphant,Queerama
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.041569,0.008708,0.006937,0.005595,0.000000,0.006456,0.059202,0.000000,0.000000,...,0.0,0.051110,0.028298,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
Jumanji,0.041569,1.000000,0.000000,0.065065,0.000000,0.000000,0.000000,0.165721,0.028302,0.011462,...,0.0,0.000000,0.039299,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
Grumpier Old Men,0.008708,0.000000,1.000000,0.035846,0.010906,0.000000,0.033363,0.000000,0.000000,0.000000,...,0.0,0.099628,0.000000,0.0,0.0,0.0,0.106819,0.000000,0.0,0.0
Waiting to Exhale,0.006937,0.065065,0.035846,1.000000,0.093741,0.003806,0.063686,0.027484,0.000000,0.000000,...,0.0,0.135728,0.000000,0.0,0.0,0.0,0.121701,0.037622,0.0,0.0
Father of the Bride Part II,0.005595,0.000000,0.010906,0.093741,1.000000,0.000000,0.038016,0.000000,0.000000,0.000000,...,0.0,0.064015,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [34]:
def genre_recommendations(target_title, matrix, items, k=10):
    recom_idx = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    recom_title = items.iloc[recom_idx, :].title.values
    recom_genre = items.iloc[recom_idx, :].genres.values
    target_title_list = np.full(len(range(k)), target_title)
    target_genre_list = np.full(len(range(k)), items[items.title == target_title].genres.values)
    d = {
        'target_title':target_title_list,
        'target_genre':target_genre_list,
        'recom_title' : recom_title,
        'recom_genre' : recom_genre
    }
    return pd.DataFrame(d)

In [35]:
genre_recommendations('The Dark Knight Rises', cosine_sim_df, movie_data)

,target_title,target_genre,recom_title,recom_genre
0,The Dark Knight Rises,Action Crime Drama Thriller,The Dark Knight,Drama Action Crime Thriller
1,The Dark Knight Rises,Action Crime Drama Thriller,The Burglar,Crime Drama
2,The Dark Knight Rises,Action Crime Drama Thriller,Batman Begins,Action Crime Drama
3,The Dark Knight Rises,Action Crime Drama Thriller,Batman & Robin,Action Crime Fantasy
4,The Dark Knight Rises,Action Crime Drama Thriller,Batman,Fantasy Action
5,The Dark Knight Rises,Action Crime Drama Thriller,Raffles,Adventure Comedy Crime Drama History Romance T...
6,The Dark Knight Rises,Action Crime Drama Thriller,Hero at Large,Action Comedy Drama
7,The Dark Knight Rises,Action Crime Drama Thriller,DC Showcase: Catwoman,Action Adventure Animation Science Fiction
8,The Dark Knight Rises,Action Crime Drama Thriller,DC Super Hero Girls: Hero of the Year,Animation
9,The Dark Knight Rises,Action Crime Drama Thriller,Batman Returns,Action Fantasy


# Case 2

genres : 영화 장르

keywords : 영화의 키워드

original_language : 영화 언어

title : 제목

vote_average : 평점 평균

vote_count : 평점 카운트

popularity : 인기도

overview : 개요 설명

### 데이터 불러오기

In [110]:
data=pd.read_csv('tmdb_5000_movies.csv')

In [111]:
data.head(2)

,budget,genres,homepage,...,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,...,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,...,Pirates of the Caribbean: At World's End,6.9,4500


### 전처리 -> 사용할 데이터만 추출

In [112]:
data=data[['id','genres','vote_average','vote_count','popularity','title','keywords','overview']]

영화평점에서 평균평점에 대해 불공정한 지표가 있을 수 있음. 왜냐하면 count수가 적을수록 높은평점을 받을 가능성이 존재함

그래서 이런 불공정을 처리하기 위해 imdb에서 처리한 방법 weighted rating 이용

*가중 정격(WR) = (v / (v+m)) × R + (m / (v+m)) × C 여기서:

r : 개별 영화 평점

v : 개별 영화에 평점을 투표한 횟수

m : 250위 안에 들어야 하는 최소 투표 (정하기 나름인듯. 난 500이라고 하면 500으로 해도 되고.)

c : 전체 영화에 대한 평균 평점

In [113]:
m=data['vote_count'].quantile(0.9)
data=data.loc[data['vote_count']>=m]

In [114]:
data.head()

,id,genres,vote_average,...,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,...,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,...,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,...,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,...,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,...,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


def weighted_rating이라는 함수를 만들고 m값, C값을 활용해 계산을 한 뒤 그 결과를 return합니다.

그리고 그 결과를 data['score']라는 컬럼에 넣습니다.

In [115]:
C=data['vote_average'].mean()

In [116]:
print(C)
print(m)

6.9629937629937615
1838.4000000000015


In [117]:
def weighted_rating(x,m=m,C=C):
    v=x['vote_count']
    R=x['vote_average']
    
    return (v / (v+m) * R) + (m / (m+v) * C)

In [118]:
data['score']=data.apply(weighted_rating,axis=1)

In [119]:
data.head()

,id,genres,vote_average,...,keywords,overview,score
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,...,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,...,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",6.918271
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,...,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,6.493333
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,...,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,7.492998
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,...,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",6.500396


In [120]:
data.shape

(481, 9)

In [121]:
data[['genres','keywords']].head(2)

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."


 Python list안에 dictionary가 포함된 구조입니다.

또한, 저게 현재 문자열(string) 형태로 되어있어서 변환이 필요한데요. 이는 파이썬(Python) 패키지 ast안에 있는 literal_eval을 사용하면됩니다.

In [122]:
data['genres']=data['genres'].apply(literal_eval)
data['keywords']=data['keywords'].apply(literal_eval)

In [123]:
data[['genres','keywords']].head(2)

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."


저 dictionary식으로 되어 있는 데이터에서 우리는 name만 뽑아내야합니다. 저희에게 저 id 값은 필요가 없기 때문입니다.

data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))을 통해 

dict 형태 -> list 형태 -> 띄어쓰기로 이루어진 str로 변경해줍니다.

In [124]:
data['genres']=data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))
data['keywords']=data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [125]:
data.head(2)

,id,genres,vote_average,...,keywords,overview,score
0,19995,Action Adventure Fantasy Science Fiction,7.2,...,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,Adventure Fantasy Action,6.9,...,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.918271


In [126]:
data.genres.head(2)

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
Name: genres, dtype: object

## 콘텐츠 기반 필터링 추천(content based filtering)하기
이제 전처리가 끝나고 본격적으로 content based filtering을 진행합니다.

여기서는 비슷한 영화를 추천해주는 것이기 때문에 영화 장르를 활용해봅니다.

현재 장르는 위 전처리 과정을 겪고나서 띄어쓰기를 구분자로 한 문자열로 되어있습니다.

이 문자열을 숫자로 바꾸어 벡터화 시킵니다! Python scikit learn에 있는 CountVectorizer를 이용하면 좋습니다.

In [127]:
count_vector = CountVectorizer(ngram_range=(1,3))

In [128]:
c_vector_genres=count_vector.fit_transform(data['genres'])

In [129]:
c_vector_genres.shape

(481, 364)

코사인 유사도를 이용해서 유사도 높은 순서대로 argsort로 정렬

In [130]:
#코사인 유사도를 구한 벡터를 미리 저장
genre_c_sim=cosine_similarity(c_vector_genres,c_vector_genres).argsort()[:,::-1]

In [131]:
genre_c_sim.shape

(481, 481)

In [132]:
data

,id,genres,vote_average,...,keywords,overview,score
0,19995,Action Adventure Fantasy Science Fiction,7.2,...,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,Adventure Fantasy Action,6.9,...,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.918271
2,206647,Action Adventure Crime,6.3,...,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...,6.493333
3,49026,Action Crime Drama Thriller,7.6,...,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,7.492998
4,49529,Action Adventure Science Fiction,6.1,...,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",6.500396
...,...,...,...,...,...,...,...
4291,176,Horror Mystery Crime,7.2,...,shotgun based on short film sadist pistol chai...,Obsessed with teaching his victims the value o...,7.091679
4300,500,Crime Thriller,8.0,...,traitor jewelry psychopath thief heist betraya...,A botched robbery indicates a police informant...,7.655593
4302,429,Western,8.1,...,bounty hunter refugee gold anti hero gallows h...,While the Civil War rages between the Union an...,7.596247
4337,103,Crime Drama,8.0,...,vietnam veteran taxi obsession drug dealer nig...,A mentally unstable Vietnam War veteran works ...,7.564085


In [133]:
data.columns

Index(['id', 'genres', 'vote_average', 'vote_count', 'popularity', 'title', 'keywords', 'overview', 'score'], dtype='object')

기본 로직은

1. 영화 제목이 들어오면 영화 제목을 가지고 있는 index를 뽑아낸다.

2. 코사인 유사도 중 영화 제목 인덱스에 해당하는 값에서 추천 개수만큼 뽑아낸다.

3. 본인은 제외

4. imdb weighted rating을 적용한 score 기반으로 정렬

5. 이를 Python DataFrame으로 만들고 return

In [136]:
def get_recommend_movie_list(df,movie_title,top=30):
    #특정 영화와 비슷한 영화를추천해야 하기에 '특정 영화' 정보를 추출
    target_movie_index=df[df['title']==movie_title].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보 추출
    sim_index=genre_c_sim[target_movie_index,:top].reshape(-1)
    
    #본인 제외
    sim_index=sim_index[sim_index != target_movie_index]
    
    #data frame로 만들고 vote_count로 정렬한뒤 return
    result=df.iloc[sim_index].sort_values('score',ascending=False)[:10]
    return result

In [138]:
target_movie_index=data[data['title']=='Avatar'].index.values

In [139]:
target_movie_index

array([0], dtype=int64)

In [140]:
sim_index=genre_c_sim[target_movie_index,:30].reshape(-1)

In [141]:
sim_index

array([  0,  13,  42,  88, 103,  18,  17,  34,   9,  26,  62, 166, 163,
        54,  32,  43, 132,  31,  51, 168,  53, 177, 127,  71,  76, 125,
        86, 244, 234,  27], dtype=int64)

In [142]:
sim_index=sim_index[sim_index != target_movie_index]

In [143]:
sim_index

array([ 13,  42,  88, 103,  18,  17,  34,   9,  26,  62, 166, 163,  54,
        32,  43, 132,  31,  51, 168,  53, 177, 127,  71,  76, 125,  86,
       244, 234,  27], dtype=int64)

In [145]:
result=data.iloc[sim_index].sort_values('score',ascending=False)[:10]

In [146]:
result

,id,genres,vote_average,...,keywords,overview,score
85,100402,Action Adventure Science Fiction,7.6,...,washington d.c. future shield marvel comic sup...,After the cataclysmic events in New York with ...,7.445960
46,127585,Action Adventure Fantasy Science Fiction,7.5,...,1970s mutant time travel marvel comic based on...,The ultimate X-Men ensemble fights a war for t...,7.374564
47,54138,Action Adventure Science Fiction,7.4,...,spacecraft friendship sequel futuristic space ...,When the crew of the Enterprise is called back...,7.271589
19,122917,Action Adventure Fantasy,7.1,...,corruption elves dwarves orcs middle-earth (to...,Immediately after the events of The Desolation...,7.061828
182,102899,Science Fiction Action Adventure,7.0,...,marvel comic superhero based on comic book aft...,Armed with the astonishing ability to shrink i...,6.991186
426,70160,Science Fiction Adventure Fantasy,6.9,...,hallucination dystopia female protagonist bow ...,Every year in the ruins of what was once North...,6.910254
126,76338,Action Adventure Fantasy,6.8,...,marvel comic superhero based on comic book hos...,Thor fights to restore order across the cosmos...,6.845447
31,68721,Action Adventure Science Fiction,6.8,...,terrorist war on terror tennessee malibu marve...,When Tony Stark's world is torn apart by a for...,6.828151
507,602,Action Adventure Science Fiction,6.7,...,spacecraft patriotism countdown independence i...,"On July 2, a giant alien mothership enters orb...",6.794831
30,558,Action Adventure Fantasy,6.7,...,dual identity love of one's life pizza boy mar...,Peter Parker is going through a major identity...,6.778496


In [137]:
get_recommend_movie_list(data,'The Dark Knight Rises')

,id,genres,vote_average,...,keywords,overview,score
65,155,Drama Action Crime Thriller,8.2,...,dc comics crime fighter secret identity scarec...,Batman raises the stakes in his war on crime. ...,8.035690
2091,274,Crime Drama Thriller,8.1,...,based on novel psychopath horror suspense seri...,"FBI trainee, Clarice Starling ventures into a ...",7.767228
2760,264644,Drama Thriller,8.1,...,based on novel carpet isolation kidnapping imp...,Jack is a young boy of 5 years old who has liv...,7.645138
351,1422,Drama Thriller Crime,7.9,...,undercover boston police friends mafia underco...,"To take down South Boston's Irish Mafia, the p...",7.621146
1850,111,Action Crime Drama Thriller,8.0,...,miami corruption capitalism cuba prohibition b...,After getting a green card in exchange for ass...,7.601698
4337,103,Crime Drama,8.0,...,vietnam veteran taxi obsession drug dealer nig...,A mentally unstable Vietnam War veteran works ...,7.564085
1051,146233,Drama Thriller Crime,7.9,...,pennsylvania kidnapping maze vigilante rural s...,When Keller Dover's daughter and her friend go...,7.550121
828,24,Action Crime,7.7,...,japan coma martial arts kung fu underworld yak...,An assassin is shot at the altar by her ruthle...,7.500378
3701,641,Crime Drama,7.9,...,drug addiction junkie heroin speed diet unsoci...,The hopes and dreams of four ambitious people ...,7.497657
1829,6977,Crime Drama Thriller,7.7,...,texas drug traffic hitman united states–mexico...,"Llewelyn Moss stumbles upon dead bodies, $2 mi...",7.420140


# Case 3 : 사용자 기반

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [64]:
data = pd.read_csv('ratings_small.csv')

In [65]:
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [66]:
data = data.pivot_table('rating', index = 'userId', columns = 'movieId')

In [67]:
data.shape

(671, 9066)

In [68]:
ratings = pd.read_csv('ratings_small.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

In [69]:
movies.rename(columns = {'id': 'movieId'}, inplace = True)

In [70]:

ratings_movies = pd.merge(ratings, movies, on = 'movieId')

In [71]:
ratings_movies.head(1)

,userId,movieId,rating,timestamp,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,1,2105,4.0,1260759139,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296


In [72]:
ratings_movies.shape

(18571, 23)

In [73]:
data = ratings_movies.pivot_table('rating', index = 'userId', columns = 'title').fillna(0)

In [78]:
rating_data = pd.read_csv('ratings.csv')
movie_data = pd.read_csv('movies.csv')

In [79]:
rating_data.head(2)

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676


In [80]:
movie_data.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [81]:
rating_data.drop('timestamp',axis=1,inplace=True)
rating_data.head(2)

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5


In [82]:
user_movie_rating=pd.merge(rating_data,movie_data,on='movieId')
user_movie_rating.head(2)

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,Jumanji (1995),Adventure|Children|Fantasy


In [83]:
user_movie_rating=user_movie_rating.pivot_table('rating',index='userId',columns='title')

In [84]:
user_movie_rating.head(5)

title,"""Great Performances"" Cats (1998)",$5 a Day (2008),'71 (2014),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'R Xmas (2001),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",...,[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nos amours (1983),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
user_movie_rating.fillna(0,inplace=True)
user_movie_rating.head(3)

title,"""Great Performances"" Cats (1998)",$5 a Day (2008),'71 (2014),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'R Xmas (2001),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",...,[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nos amours (1983),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
user_based_collabor=cosine_similarity(user_movie_rating)
user_based_collabor

array([[1.        , 0.10291644, 0.26119811, ..., 0.0731671 , 0.06134694,
        0.07374516],
       [0.10291644, 1.        , 0.18012391, ..., 0.0727862 , 0.09677127,
        0.10142109],
       [0.26119811, 0.18012391, 1.        , ..., 0.15637485, 0.14209048,
        0.03905254],
       ...,
       [0.0731671 , 0.0727862 , 0.15637485, ..., 1.        , 0.05162821,
        0.        ],
       [0.06134694, 0.09677127, 0.14209048, ..., 0.05162821, 1.        ,
        0.23723644],
       [0.07374516, 0.10142109, 0.03905254, ..., 0.        , 0.23723644,
        1.        ]])

In [88]:
user_based_collabor=pd.DataFrame(data=user_based_collabor,index=user_movie_rating.index,columns=user_movie_rating.index)

In [89]:
user_based_collabor.head()

userId,1,2,3,4,5,6,7,8,9,10,...,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.102916,0.261198,0.029091,0.139199,0.029488,0.112060,0.060641,0.062589,0.136902,...,0.111705,0.024332,0.068812,0.070645,0.072708,0.199134,0.184821,0.073167,0.061347,0.073745
2,0.102916,1.000000,0.180124,0.064014,0.140042,0.092464,0.160006,0.103785,0.000000,0.101933,...,0.032184,0.000000,0.101155,0.029610,0.056683,0.097934,0.131996,0.072786,0.096771,0.101421
3,0.261198,0.180124,1.000000,0.057323,0.156755,0.063552,0.207623,0.087128,0.053957,0.273834,...,0.067894,0.065551,0.115196,0.140032,0.094147,0.211351,0.244160,0.156375,0.142090,0.039053
4,0.029091,0.064014,0.057323,1.000000,0.300828,0.040654,0.073242,0.307392,0.042953,0.025546,...,0.073765,0.050096,0.414432,0.000000,0.291746,0.078976,0.230812,0.000000,0.230159,0.024401
5,0.139199,0.140042,0.156755,0.300828,1.000000,0.234223,0.169685,0.304162,0.000000,0.138848,...,0.088054,0.000000,0.420953,0.053778,0.362804,0.166011,0.339279,0.090617,0.365986,0.140729


In [91]:
def get_user_based_collabor(userId):
    return user_based_collabor[userId].sort_values(ascending=False)[:6]

In [92]:
get_user_based_collabor(1)

userId
1       1.000000
2595    0.415359
6364    0.389495
1748    0.381045
6622    0.380857
5366    0.375403
Name: 1, dtype: float64

# 사용자 기반 2

In [93]:
import pandas as pd
import numpy as np

ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

pd.set_option('display.max_columns', 6)
pd.set_option('display.width', 300)

movie_ratings = pd.merge(ratings, movies, on='movieId')
print(movie_ratings)

         userId  movieId  rating   timestamp                                              title                      genres
0             1        2     3.5  1112486027                                     Jumanji (1995)  Adventure|Children|Fantasy
1             5        2     3.0   851527569                                     Jumanji (1995)  Adventure|Children|Fantasy
2            13        2     3.0   849082742                                     Jumanji (1995)  Adventure|Children|Fantasy
3            29        2     3.0   835562174                                     Jumanji (1995)  Adventure|Children|Fantasy
4            34        2     3.0   846509384                                     Jumanji (1995)  Adventure|Children|Fantasy
...         ...      ...     ...         ...                                                ...                         ...
1048570    7066    88572     1.5  1417484236                             Fred: The Movie (2010)                      Comedy
1048571 

In [94]:
title_user = movie_ratings.pivot_table('rating', index='userId', columns='title')
title_user.fillna(0, inplace=True)

In [101]:
from sklearn.metrics.pairwise import cosine_similarity

# 유저와 유저 간의 유사도
user_based_collab = cosine_similarity(title_user, title_user)
user_based_collab = pd.DataFrame(user_based_collab, index=title_user.index, columns=title_user.index)

print(user_based_collab[1].sort_values(ascending=False)[:10])

userId
1       1.000000
2595    0.415359
6364    0.389495
1748    0.381045
6622    0.380857
5366    0.375403
6078    0.352713
1839    0.349549
147     0.348325
4421    0.348082
Name: 1, dtype: float64


In [99]:
# 가장 유사한 유저를 뽑아서 해당 유저가 본 영화 추천
user = user_based_collab[1].sort_values(ascending=False)[:10].index[1]
print(title_user.loc[user].sort_values(ascending=False))

title
Taxi Driver (1976)                                                  5.0
Goodfellas (1990)                                                   5.0
12 Angry Men (1957)                                                 5.0
King Kong (1933)                                                    5.0
Koyaanisqatsi (a.k.a. Koyaanisqatsi: Life Out of Balance) (1983)    5.0
                                                                   ... 
Out Cold (2001)                                                     0.0
Our Town (1940)                                                     0.0
Our Song (2000)                                                     0.0
Our Relations (1936)                                                0.0
"Great Performances" Cats (1998)                                    0.0
Name: 2595, Length: 14021, dtype: float64


In [97]:
# 1번 유저와 유사한 유저들 9명을 뽑아서, 그 유저들이 어떤 영화에 대해서 부여한 평점에
# 유사도만큼의 가중치를 부여해서 이걸 토대로 1번 유저가 부여할 평점을 계산/예측
# 가중치 -> 유저 9명 유사도의 합 중에서 해당 유저가 차지하는 유사도
user_index_list = user_based_collab[1].sort_values(ascending=False)[:10].index.tolist()
user_weight_list = user_based_collab[1].sort_values(ascending=False)[:10].tolist()